In [1]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="xgb", 
           entity="benchmark-nlp",
           name='fake news datasets') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os
#Move back to the root directory of the project
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42
OPT_ITER=20

## Load data

In [5]:
dl = DataLoader(['fake_news'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization

train_gossipcop, val_gossipcop, _ = data_splitter(data['gossipcop'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)


100%|█████████████████████████████████████████████████████████████████████████████| 5323/5323 [00:41<00:00, 128.64it/s]


18590 rows preprocessed in 97.84012413024902 seconds


In [6]:
train_coaid, val_coaid, _ = data_splitter(data['CoAID'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 test_split=0.25,
                                 val_split=0.2,
                                 seed=SEED)
train_liar, val_liar, _ = data_splitter(data['liar'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

In [7]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 17.9 s
Wall time: 34.1 s


In [8]:
embedded_train_gossipcop = fasttext.generate_sentence_embeddings(train_gossipcop['text'])
embedded_val_gossipcop = fasttext.generate_sentence_embeddings(val_gossipcop['text'])
embedded_train_gossipcop['label'] = train_gossipcop['label'].to_list()
embedded_val_gossipcop['label'] = val_gossipcop['label'].to_list()

Starting to generate sentence embeddings


100%|████████████████████████████████████████████████████████████████████████████| 11897/11897 [04:10<00:00, 47.50it/s]


Starting to generate sentence embeddings


100%|██████████████████████████████████████████████████████████████████████████████| 2975/2975 [01:05<00:00, 45.41it/s]


In [17]:
embedded_train_coaid = fasttext.generate_sentence_embeddings(train_coaid['text'])
embedded_val_coaid = fasttext.generate_sentence_embeddings(val_coaid['text'])
embedded_train_coaid['label'] = train_coaid['label'].to_list()
embedded_val_coaid['label'] = val_coaid['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 3273/3273 [00:09<00:00, 351.32it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████████| 819/819 [00:02<00:00, 373.09it/s]


In [9]:
embedded_train_liar = fasttext.generate_sentence_embeddings(train_liar['text'])
embedded_val_liar = fasttext.generate_sentence_embeddings(val_liar['text'])
embedded_train_liar['label'] = train_liar['label'].to_list()
embedded_val_liar['label'] = val_liar['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 10269/10269 [00:36<00:00, 279.58it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 1284/1284 [00:03<00:00, 407.03it/s]


## Hyperopt

In [9]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/xgb_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [10]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'gamma': {'min': 0, 'max': 1, 'distribution': 'uniform'},
  'n_estimators': {'min': 10, 'max': 150, 'distribution': 'int_uniform'},
  'learning_rate': {'min': 0.001, 'max': 0.1, 'distribution': 'uniform'},
  'max_depth': {'min': 0, 'max': 10, 'distribution': 'int_uniform'},
  'random_state': {'value': 42}}}

#### gossipcop

In [ ]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_gossipcop' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_gossipcop, #Change here
          val=val_gossipcop): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

In [ ]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_gossipcop' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_gossipcop, #CHANGE HERE
          val=embedded_val_gossipcop): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

####  CoAiD



In [12]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_coaid' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_coaid, #Change here
          val=val_coaid): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:28:24] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 3jaychrx
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/3jaychrx


[codecarbon WARNING @ 13:28:28] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 2rqk7f7e with config:
wandb: 	gamma: 0.803627837598068
wandb: 	learning_rate: 0.01487720120326014
wandb: 	max_depth: 3
wandb: 	n_estimators: 63
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.89538
AUC-PC,0.96725
accuracy,0.89133
f1 macro,0.77734


wandb: Agent Starting Run: n4skxv8u with config:
wandb: 	gamma: 0.5423461612102266
wandb: 	learning_rate: 0.01077590317175046
wandb: 	max_depth: 1
wandb: 	n_estimators: 17
wandb: 	random_state: 42


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: soyl99po with config:
wandb: 	gamma: 0.8432000119288168
wandb: 	learning_rate: 0.08892216573945395
wandb: 	max_depth: 10
wandb: 	n_estimators: 30
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.95124
AUC-PC,0.98919
accuracy,0.91453
f1 macro,0.8303


wandb: Agent Starting Run: xoqjsi8y with config:
wandb: 	gamma: 0.01257341857072758
wandb: 	learning_rate: 0.02825924609698786
wandb: 	max_depth: 0
wandb: 	n_estimators: 93
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.5
AUC-PC,0.83394
accuracy,0.83394
f1 macro,0.45473


0.0034603223140008657

In [25]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_coaid' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_coaid, #CHANGE HERE
          val=embedded_val_coaid): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 19:10:39] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 2lkcm8ev
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/2lkcm8ev


[codecarbon WARNING @ 19:10:42] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: 3ttvjndu with config:
wandb: 	gamma: 0.9229902247386036
wandb: 	learning_rate: 0.08225241931294472
wandb: 	max_depth: 4
wandb: 	n_estimators: 17
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.9571
AUC-PC,0.98967
accuracy,0.91331
f1 macro,0.82363


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: tho4bbwv with config:
wandb: 	gamma: 0.17997929845326832
wandb: 	learning_rate: 0.06510934539035863
wandb: 	max_depth: 2
wandb: 	n_estimators: 34
wandb: 	random_state: 42


...

wandb: Agent Starting Run: gnx7ksq8 with config:
wandb: 	gamma: 0.4155623020813912
wandb: 	learning_rate: 0.012040942563092238
wandb: 	max_depth: 7
wandb: 	n_estimators: 18
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.93831
AUC-PC,0.98493
accuracy,0.89866
f1 macro,0.80967


wandb: Agent Starting Run: jirq5u48 with config:
wandb: 	gamma: 0.98885470736457
wandb: 	learning_rate: 0.08222559858376109
wandb: 	max_depth: 8
wandb: 	n_estimators: 59
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.96873
AUC-PC,0.99349
accuracy,0.94017
f1 macro,0.88621


0.0032671196163521665

#### liar

In [15]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_liar' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_liar, #Change here
          val=val_liar): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

Create sweep with ID: 49qir2ww
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/49qir2ww


[codecarbon WARNING @ 16:10:22] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 16:10:26] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: fbed0h82 with config:
wandb: 	gamma: 0.12353902011952922
wandb: 	learning_rate: 0.015059595514406703
wandb: 	max_depth: 4
wandb: 	n_estimators: 114
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.2134
f1 macro,0.15171


wandb: Agent Starting Run: hzu6oeeo with config:
wandb: 	gamma: 0.10618271088403695
wandb: 	learning_rate: 0.050215583347495686
wandb: 	max_depth: 6
wandb: 	n_estimators: 89
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.24377
f1 macro,0.19328


...

wandb: Agent Starting Run: zqsghzn5 with config:
wandb: 	gamma: 0.8103916395021523
wandb: 	learning_rate: 0.06284145643864351
wandb: 	max_depth: 3
wandb: 	n_estimators: 105
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.2204
f1 macro,0.17071


wandb: Agent Starting Run: 0mvuqiug with config:
wandb: 	gamma: 0.8765847423948667
wandb: 	learning_rate: 0.06480873617175187
wandb: 	max_depth: 4
wandb: 	n_estimators: 43
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.2095
f1 macro,0.15392


0.0032428502195058343

In [16]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_liar' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_liar, #CHANGE HERE
          val=embedded_val_liar): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 16:32:05] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: ugwz4xz6
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/ugwz4xz6


[codecarbon WARNING @ 16:32:09] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: n0wdjrhk with config:
wandb: 	gamma: 0.6439190854821173
wandb: 	learning_rate: 0.09177557113871249
wandb: 	max_depth: 1
wandb: 	n_estimators: 26
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.23598
f1 macro,0.14996


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: bfz0howy with config:
wandb: 	gamma: 0.16484690958326187
wandb: 	learning_rate: 0.026709659965655785
wandb: 	max_depth: 7
wandb: 	n_estimators: 85
wandb: 	random_state: 42


...

wandb: Agent Starting Run: yntl7jgo with config:
wandb: 	gamma: 0.5424137044218664
wandb: 	learning_rate: 0.032824904720252825
wandb: 	max_depth: 1
wandb: 	n_estimators: 84
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.23209
f1 macro,0.14897


wandb: Agent Starting Run: q8urcvva with config:
wandb: 	gamma: 0.9801801933296108
wandb: 	learning_rate: 0.04768416160360133
wandb: 	max_depth: 4
wandb: 	n_estimators: 144
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.25078
f1 macro,0.23087


0.011878413049547929